# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 8:30AM,258659,10,9228665,Eywa Pharma Inc.,Released
1,Mar 6 2023 8:30AM,258656,10,PRF-54686,Bio-PRF,Released
2,Mar 6 2023 8:30AM,258656,10,PRF-54691,Bio-PRF,Released
3,Mar 6 2023 8:30AM,258656,10,PRF-54694,Bio-PRF,Released
4,Mar 6 2023 8:30AM,258656,10,PRF-54697,Bio-PRF,Released
5,Mar 6 2023 8:30AM,258657,10,Enova-11461,Emerginnova,Released
6,Mar 6 2023 8:30AM,258657,10,Enova-11462,Emerginnova,Executing
7,Mar 3 2023 3:36PM,258660,22,30223,"NBTY Global, Inc.",Executing
8,Mar 3 2023 2:32PM,258654,12,9228146,HealthCaps LLC,Executing
9,Mar 3 2023 2:15PM,258651,10,0086473512,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,258656,Released,4
12,258657,Executing,1
13,258657,Released,1
14,258659,Released,1
15,258660,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258654,1.0,NaN
258656,NaN,4.0
258657,1.0,1.0
258659,NaN,1.0
258660,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258617,0.0,1.0
258618,0.0,1.0
258623,0.0,1.0
258624,0.0,2.0
258627,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258617,0,1
258618,0,1
258623,0,1
258624,0,2
258627,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258617,0,1
1,258618,0,1
2,258623,0,1
3,258624,0,2
4,258627,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258617,,1
1,258618,,1
2,258623,,1
3,258624,,2
4,258627,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 8:30AM,258659,10,Eywa Pharma Inc.
1,Mar 6 2023 8:30AM,258656,10,Bio-PRF
5,Mar 6 2023 8:30AM,258657,10,Emerginnova
7,Mar 3 2023 3:36PM,258660,22,"NBTY Global, Inc."
8,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC
9,Mar 3 2023 2:15PM,258651,10,ISDIN Corporation
10,Mar 3 2023 2:05PM,258649,15,"Person & Covey, Inc."
18,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.
19,Mar 3 2023 11:06AM,258628,12,Hush Hush
20,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 6 2023 8:30AM,258659,10,Eywa Pharma Inc.,,1
1,Mar 6 2023 8:30AM,258656,10,Bio-PRF,,4
2,Mar 6 2023 8:30AM,258657,10,Emerginnova,1,1
3,Mar 3 2023 3:36PM,258660,22,"NBTY Global, Inc.",1,
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,1,
5,Mar 3 2023 2:15PM,258651,10,ISDIN Corporation,,1
6,Mar 3 2023 2:05PM,258649,15,"Person & Covey, Inc.",7,1
7,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,,1
8,Mar 3 2023 11:06AM,258628,12,Hush Hush,1,
9,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 8:30AM,258659,10,Eywa Pharma Inc.,1,
1,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4,
2,Mar 6 2023 8:30AM,258657,10,Emerginnova,1,1
3,Mar 3 2023 3:36PM,258660,22,"NBTY Global, Inc.",,1
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,,1
5,Mar 3 2023 2:15PM,258651,10,ISDIN Corporation,1,
6,Mar 3 2023 2:05PM,258649,15,"Person & Covey, Inc.",1,7
7,Mar 3 2023 1:38PM,258647,16,Sartorius Stedim Filters Inc.,1,
8,Mar 3 2023 11:06AM,258628,12,Hush Hush,,1
9,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 8:30AM,258659,10,Eywa Pharma Inc.,1,
1,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4,
2,Mar 6 2023 8:30AM,258657,10,Emerginnova,1,1
3,Mar 3 2023 3:36PM,258660,22,"NBTY Global, Inc.",,1
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 8:30AM,258659,10,Eywa Pharma Inc.,1.0,NaN
1,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4.0,NaN
2,Mar 6 2023 8:30AM,258657,10,Emerginnova,1.0,1.0
3,Mar 3 2023 3:36PM,258660,22,"NBTY Global, Inc.",NaN,1.0
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 8:30AM,258659,10,Eywa Pharma Inc.,1.0,0.0
1,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4.0,0.0
2,Mar 6 2023 8:30AM,258657,10,Emerginnova,1.0,1.0
3,Mar 3 2023 3:36PM,258660,22,"NBTY Global, Inc.",0.0,1.0
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1034623,7.0,1.0
12,517282,0.0,2.0
15,258649,1.0,7.0
16,258647,1.0,0.0
19,1034482,5.0,0.0
20,258627,1.0,0.0
22,258660,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1034623,7.0,1.0
1,12,517282,0.0,2.0
2,15,258649,1.0,7.0
3,16,258647,1.0,0.0
4,19,1034482,5.0,0.0
5,20,258627,1.0,0.0
6,22,258660,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,7.0,1.0
1,12,0.0,2.0
2,15,1.0,7.0
3,16,1.0,0.0
4,19,5.0,0.0
5,20,1.0,0.0
6,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,0.0
2,15,Released,1.0
3,16,Released,1.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Executing,1.0,2.0,7.0,0.0,0.0,0.0,1.0
Released,7.0,0.0,1.0,1.0,5.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Executing,1.0,2.0,7.0,0.0,0.0,0.0,1.0
1,Released,7.0,0.0,1.0,1.0,5.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Executing,1.0,2.0,7.0,0.0,0.0,0.0,1.0
1,Released,7.0,0.0,1.0,1.0,5.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()